# **Preparacion DF**

In [264]:
# Importando librerías necesarias
import pandas as pd
import matplotlib as plt

In [265]:
df = pd.read_csv('streaming.csv')

In [266]:
print(df.shape) 
print(df.info())

(22998, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             22998 non-null  object 
 1   type           22998 non-null  object 
 2   title          22998 non-null  object 
 3   director       14739 non-null  object 
 4   cast           17677 non-null  object 
 5   country        11499 non-null  object 
 6   date_added     13444 non-null  object 
 7   release_year   22998 non-null  int64  
 8   rating         22998 non-null  object 
 9   duration_int   22998 non-null  int64  
 10  duration_type  22516 non-null  object 
 11  listed_in      22998 non-null  object 
 12  description    22994 non-null  object 
 13  userId         22998 non-null  int64  
 14  score          22998 non-null  float64
 15  timestamp      22998 non-null  int64  
 16  mean_score     22998 non-null  float64
dtypes: float64(2), int64(4), object(11)
me

In [267]:
df.head()

,id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,userId,score,timestamp,mean_score
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,543,5.0,1059596582,3.467131
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110,min,"drama, international",a metro family decides to fight a cyber crimin...,321,3.0,1181451581,3.548682
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74,min,"action, drama, suspense",after a man discovers his wife is cheating on ...,582,5.0,1316843370,3.500000
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69,min,documentary,"pink breaks the mold once again, bringing her ...",151,1.5,1081454081,3.538055
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45,min,"drama, fantasy",teenage matt banting wants to work with a famo...,24,5.0,979972491,3.478992


In [268]:
#renombro columna
df = df.rename(columns={'listed_in': 'genero'})

In [269]:
#seleccionar campos de interes
df = df[['id', 'title', 'genero', 'score', 'userId']]
# Reestablezco índices
df = df.reset_index(drop=True)

In [270]:
df.to_csv(r'streaming_ML.csv', index = False)

In [271]:
print(df.shape) 
df.info()

(22998, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      22998 non-null  object 
 1   title   22998 non-null  object 
 2   genero  22998 non-null  object 
 3   score   22998 non-null  float64
 4   userId  22998 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 898.5+ KB


In [272]:
#verificamos nulos
df.isnull().sum()

id        0
title     0
genero    0
score     0
userId    0
dtype: int64

# **Modelo de Recomendación**

In [273]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import accuracy

# **Modelo 1**

Ajuste data para algoritmo

In [274]:
N_filas = 10000 
# Creando un objeto Reader que interpreta los datos de calificación en una escala de 1 a 5.
reader = Reader(rating_scale=(1, 5))
# Conviertiendo el DataFrame en un formato que puede ser procesado por el modelo.
data = Dataset.load_from_df(df[["userId", "id", "score"]][:N_filas], reader)

In [275]:
train, test = train_test_split(data, test_size=0.25)

Algoritmo, Entrenamiento

In [276]:
model = SVD()
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
model.fit(train)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9785  0.9520  0.9756  0.9687  0.0118  
MAE (testset)     0.7701  0.7497  0.7648  0.7615  0.0086  
Fit time          0.23    0.11    0.11    0.15    0.06    
Test time         0.03    0.01    0.01    0.02    0.01    


Predecimos modelo

In [277]:
predictions = model.test(test)

In [278]:
predictions[5]

Prediction(uid=150, iid='as8242', r_ui=3.0, est=3.1700514614944377, details={'was_impossible': False})

Guardamos modelo

In [279]:
import pickle
pickle.dump(model, open('model.sav','wb'))

# **Modelo 2**

In [280]:
N_filas = 10000 
# Creando un objeto Reader que interpreta los datos de calificación en una escala de 1 a 5.
reader = Reader(rating_scale=(1, 5))
# Conviertiendo el DataFrame en un formato que puede ser procesado por el modelo.
data1 = Dataset.load_from_df(df[["id", "title", "score"]][:N_filas], reader)

In [281]:
train, test = train_test_split(data1, test_size=0.25)

In [282]:
model1 = SVD()
cross_validate(model1, data1, measures=['RMSE', 'MAE'], cv=3, verbose=True)
model1.fit(train)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0543  1.0131  1.0333  1.0336  0.0168  
MAE (testset)     0.8552  0.8157  0.8337  0.8349  0.0161  
Fit time          0.10    0.10    0.11    0.11    0.01    
Test time         0.01    2.16    0.01    0.73    1.01    


In [283]:
predictions = model1.test(test)

In [284]:
predictions[5]

Prediction(uid='as7373', iid='trap', r_ui=5.0, est=3.557533333333333, details={'was_impossible': False})

In [285]:
import pickle
pickle.dump(model1, open('model1.sav','wb'))

In [ ]:
items_df = pd.read_csv('streaming.csv')

In [ ]:
#seleccionar campos de interes
items_1_df = items_df[['title']]
# Reestablezco índices
items_1_df = items_1_df.reset_index(drop=True)

In [ ]:
items_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   22998 non-null  object
dtypes: object(1)
memory usage: 179.8+ KB


In [ ]:
items_1_df.to_csv(r'streaming_1_ML.csv', index = False)

In [294]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos en un DataFrame
df = pd.read_csv('streaming_1_ML.csv')

# Crear un vectorizador TfidfVectorizer
vectorizer = TfidfVectorizer()

# Ajustar y transformar el texto de la columna "descripcion" del DataFrame
X = vectorizer.fit_transform(df['title'])

# Calcular la similitud de coseno entre los ítems
similarity_matrix = cosine_similarity(X)

# Obtener las recomendaciones para un ítem específico
def get_recommendations(titulo, k=5):
    # Obtener el índice numérico de un título específico
    title_index = df.loc[df['title'] == titulo].index[0]
    # Verificar que el índice sea menor que el número de filas de la matriz
    if title_index < similarity_matrix.shape[0]:
        # Obtener las similitudes entre el ítem y los demás ítems
        item_similarities = similarity_matrix[title_index]
        # Ordenar los índices de los ítems por su similitud con el ítem de entrada
        most_similar = item_similarities.argsort()[::-1]
        # Obtener los k ítems más similares al ítem de entrada
        top_k = most_similar[1:k+1]
        # Obtener los títulos de los ítems más similares al ítem de entrada
        top_k_titles = [df.loc[i, 'title'] for i in top_k]
        # Devolver los títulos de los ítems más similares al ítem de entrada
        return top_k_titles
    else:
        # Devolver una lista vacía si el índice es mayor que el número de filas de la matriz
        return []



In [295]:
get_recommendations('the grand seduction')

['the grand', 'the grand tour', 'five grand', 'grand hotel', 'grand army']

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.extmath import randomized_svd
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar los datos en un DataFrame
df = pd.read_csv('streaming_1_ML.csv')

# Vectorizador TfidfVectorizer con parámetros de reduccion procesamiento
vectorizer = TfidfVectorizer(min_df=10, max_df=0.5, ngram_range=(1,2))

# Ajustar y transformar el texto de la columna "descripcion" del DataFrame
X = vectorizer.fit_transform(df['title'])

# Calcular la matriz de similitud de coseno con una matriz reducida de 500x500
similarity_matrix = cosine_similarity(X[:500,:])

# Obtener la descomposición en valores singulares aleatoria de la matriz de similitud de coseno con 10 componentes
n_components = 10
U, Sigma, VT = randomized_svd(similarity_matrix, n_components=n_components)

# Construir la matriz reducida de similitud de coseno
reduced_similarity_matrix = U.dot(np.diag(Sigma)).dot(VT)

# Obtener las recomendaciones para un ítem específico
def get_recommendations(titulo, k=5):
    # Obtener el índice numérico de un título específico
    title_index = df.loc[df['title'] == titulo].index[0]
    # Verificar que el índice sea menor que el número de filas de la matriz
    if title_index < reduced_similarity_matrix.shape[0]:
        # Obtener las similitudes entre el ítem y los demás ítems
        item_similarities = reduced_similarity_matrix[title_index]
        # Ordenar los índices de los ítems por su similitud con el ítem de entrada
        most_similar = item_similarities.argsort()[::-1]
        # Obtener los k ítems más similares al ítem de entrada
        top_k = most_similar[1:k+1]
        # Obtener los títulos de los ítems más similares al ítem de entrada
        top_k_titles = [df.loc[i, 'title'] for i in top_k]
        # Devolver los títulos de los ítems más similares al ítem de entrada
        return top_k_titles
    else:
        # Devolver una lista vacía si el índice es mayor que el número de filas de la matriz
        return []


In [8]:
get_recommendations('hired gun')

["the world's most peaceful resorts",
 "the world's most breathtaking waterfalls",
 "the world's most beautiful beaches",
 'the world wars',
 'the night eats the world']

In [6]:
import pickle
# Guardar el modelo con pickle
with open('reduced_similarity_matrix.sav', 'wb') as f:
    pickle.dump(reduced_similarity_matrix, f)

In [ ]:
# Cargar el modelo con pickle
with open('reduced_similarity_matrix.sav', 'rb') as f:
    reduced_similarity_matrix_loaded = pickle.load(f)